1. Crear la tabla delta “fact_ventas_final”: a. Particionada por el campo “mes”. b. Incluir el campo “monto_total” resolviendo el cálculo entre la cantidad del producto vendido y su precio unitario (tomar las dimensiones creadas).

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.gold;


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.gold.fact_ventas_final
PARTITIONED BY (mes)
AS
SELECT
    f.id_vendedor,
    v.vendedor_nombre,
    v.sucursal_nombre,
    f.id_producto,
    p.nombre AS producto_nombre,
    f.cantidad,
    f.dia,
    f.mes,
    f.ano,
    p.precio_unitario,
    (f.cantidad * p.precio_unitario) AS monto_total
FROM workspace.silver.fact_ventas f
INNER JOIN workspace.silver.dim_vendedor v
    ON f.id_vendedor = v.id_vendedor
INNER JOIN workspace.silver.dim_producto p
    ON f.id_producto = p.id_producto;


In [0]:
%sql
SELECT * 
FROM workspace.gold.fact_ventas_final
LIMIT 20;


2.
Truncar la partición de diciembre.

In [0]:
%sql
DELETE FROM workspace.gold.fact_ventas_final
WHERE mes = 12;

In [0]:
%sql
SELECT COUNT(*) AS registros_diciembre
FROM workspace.gold.fact_ventas_final
WHERE mes = 12;

3.
Actualizar la tabla descontando el 10% a la columna “monto_total” para el mes de junio.

In [0]:
%sql
UPDATE workspace.gold.fact_ventas_final
SET monto_total = monto_total * 0.9
WHERE mes = 6;

In [0]:
%sql
SELECT * 
FROM workspace.gold.fact_ventas_final 
WHERE mes = 6
limit 10

In [0]:
%sql
SELECT 
    mes,
    AVG(monto_total) AS monto_total_promedio,
    MIN(monto_total) AS monto_total_min,
    MAX(monto_total) AS monto_total_max
FROM workspace.gold.fact_ventas_final
WHERE mes = 6
GROUP BY mes;


4.
Resolver las siguientes consultas

In [0]:
%sql
-- a.Top-10 de sucursales según monto vendido ordenado de mayor a menor. Debe tener ambas columnas.
SELECT 
    sucursal_nombre,
    round(SUM(monto_total),0) AS total_vendido
FROM workspace.gold.fact_ventas_final
GROUP BY sucursal_nombre
ORDER BY total_vendido DESC
LIMIT 10;


b.
Mejores vendedores con detalle de top 3 de productos

In [0]:
%sql
-- i.Listado ordenado de mayor a menor con los mejores vendedores por “monto_total”.
SELECT 
    id_vendedor,
    vendedor_nombre,
    round(SUM(monto_total),0) AS total_vendido
FROM workspace.gold.fact_ventas_final
GROUP BY id_vendedor, vendedor_nombre
ORDER BY total_vendido DESC;


In [0]:
%sql
-- ii.Agregar los 3 productos más vendidos por cada uno con sus cantidades correspondientes.
WITH ventas_producto AS (
    SELECT
        id_vendedor,
        vendedor_nombre,
        id_producto,
        producto_nombre,
        SUM(cantidad) AS total_cantidad,
        SUM(monto_total) AS total_monto,
        ROW_NUMBER() OVER (
            PARTITION BY id_vendedor
            ORDER BY SUM(monto_total) DESC
        ) AS rn
    FROM workspace.gold.fact_ventas_final
    GROUP BY id_vendedor, vendedor_nombre, id_producto, producto_nombre
)
SELECT 
    id_vendedor,
    vendedor_nombre,
    id_producto,
    producto_nombre,
    total_cantidad,
    total_monto
FROM ventas_producto
WHERE rn <= 3
ORDER BY id_vendedor, total_monto DESC;


In [0]:
%sql
-- c) Peores 3 sucursales (menor monto vendido)
SELECT 
    sucursal_nombre,
    round(SUM(monto_total),0) AS total_vendido
FROM workspace.gold.fact_ventas_final
GROUP BY sucursal_nombre
ORDER BY total_vendido ASC
LIMIT 3;


5.
Consultar el historial de versiones de la tabla

In [0]:
%sql
DESCRIBE HISTORY workspace.gold.fact_ventas_final;


In [0]:
%sql
SELECT * 
FROM workspace.gold.fact_ventas_final TIMESTAMP AS OF '2025-09-05T16:39:24Z';



6.
Restaurar la tabla a la versión original (0)

In [0]:
%sql
RESTORE TABLE workspace.gold.fact_ventas_final TO VERSION AS OF 0;

In [0]:
%sql
DESCRIBE HISTORY workspace.gold.fact_ventas_final;

In [0]:
%sql
SELECT * 
FROM workspace.gold.fact_ventas_final
LIMIT 20;
